In [2]:
import getNewReading as gnr
import kinneretDrawGraph as kdg
import pandas as pd
import math

In [3]:
df = gnr.importReadings()
df

,level
date,
2020-12-27,-209.900
2020-12-26,-209.895
2020-12-25,-209.890
2020-12-24,-209.885
2020-12-23,-209.890
...,...
1966-11-01,-210.390
1966-10-15,-210.320
1966-10-01,-210.270


In [4]:
df['real'] = True

upsampled = df.resample('1D')
inter = upsampled.interpolate(method='cubicspline')
inter.sort_values(by='date', inplace=True, ascending=False)

In [5]:
inter.fillna(value=False, inplace=True)
inter.tail(50)

,level,real
date,,
1966-10-20,-210.337375,False
1966-10-19,-210.333597,False
1966-10-18,-210.329960,False
1966-10-17,-210.326474,False
1966-10-16,-210.323151,False
1966-10-15,-210.320000,True
1966-10-14,-210.317021,False
1966-10-13,-210.314169,False
1966-10-12,-210.311391,False


In [6]:
inter.sort_values(by='date', ascending=False, inplace=True)
inter['1day-cm'] = inter['level'].diff(periods=-1) * 100
inter['7day-cm'] = inter['level'].diff(periods=-7) * 100
inter.dropna(inplace=True)
inter

,level,real,1day-cm,7day-cm
date,,,,
2020-12-27,-209.900000,True,-0.500000,0.000000
2020-12-26,-209.895000,True,-0.500000,0.274999
2020-12-25,-209.890000,True,-0.500000,0.490148
2020-12-24,-209.885000,True,0.500000,2.000000
2020-12-23,-209.890000,True,0.500000,4.500000
...,...,...,...,...
1966-09-12,-210.101062,False,-0.927520,-5.809629
1966-09-11,-210.091787,False,-0.901988,-5.542800
1966-09-10,-210.082767,False,-0.872261,-5.246601


In [8]:
inter.sort_values('1day-cm', inplace=True)
inter

,level,real,1day-cm,7day-cm
date,,,,
1973-07-22,-210.620000,True,-6.095930,-7.000000
1973-07-21,-210.559041,False,-6.037832,0.454012
1993-06-24,-208.940000,True,-6.000000,-9.000000
1980-08-29,-209.740000,True,-6.000000,-19.818172
1980-08-28,-209.680000,True,-5.141468,-13.290472
...,...,...,...,...
2003-02-22,-212.474569,False,17.543089,51.605588
2003-02-05,-213.370000,True,20.000000,43.000000
1995-02-08,-209.410000,True,21.000000,34.000000


In [12]:
inter.describe()


,level,1day-cm,7day-cm
count,19835.000000,19835.000000,19835.000000
mean,-211.078522,0.000795,0.005103
std,1.567185,1.434881,8.671591
min,-214.870000,-6.095930,-26.097368
25%,-212.220000,-0.875357,-5.520421
50%,-210.820000,-0.280480,-2.000000
75%,-209.790000,0.489626,3.238520
max,-208.158269,23.000000,88.850369
